In [ ]:
%load_ext autoreload
%autoreload 2
from datasets import load_dataset
from transformers import AutoTokenizer

NER_TAGS_COL = 'tags'

raw_datasets = load_dataset("tner/mit_restaurant")
model_checkpoint = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

original_label_to_id = {
    "O": 0,
    "B-Rating": 1,
    "I-Rating": 2,
    "B-Amenity": 3,
    "I-Amenity": 4,
    "B-Location": 5,
    "I-Location": 6,
    "B-Restaurant_Name": 7,
    "I-Restaurant_Name": 8,
    "B-Price": 9,
    "B-Hours": 10,
    "I-Hours": 11,
    "B-Dish": 12,
    "I-Dish": 13,
    "B-Cuisine": 14,
    "I-Price": 15,
    "I-Cuisine": 16
}
original_id_to_label = {i:name for name, i in original_label_to_id.items()}

label2id = {
    "O": 0,
    "B-Rating": 1,
    "I-Rating": 2,
    "B-Amenity": 3,
    "I-Amenity": 4,
    "B-Location": 5,
    "I-Location": 6,
    "B-Restaurant_Name": 7,
    "I-Restaurant_Name": 8,
    "B-Price": 9,
    "I-Price": 10,
    "B-Hours": 11,
    "I-Hours": 12,
    "B-Dish": 13,
    "I-Dish": 14,
    "B-Cuisine": 15,
    "I-Cuisine": 16
}
id2label = {i:name for name, i in label2id.items()}

original_to_ours = {original_label_to_id[k]: label2id[k] for k in label2id}


In [ ]:
def relable(examples, original_to_ours):
    new_tags = []
    for tags in examples[NER_TAGS_COL]:
        new_tags.append([original_to_ours[item] for item in tags])
    examples[NER_TAGS_COL] = new_tags
    return examples

raw_datasets=raw_datasets.map(relable, batched=True, fn_kwargs={'original_to_ours':original_to_ours})

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
labels = raw_datasets["train"][0][NER_TAGS_COL]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))